! git clone https://github.com/Pangyk/Graph_AE.git 

! pip install pandas
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

In [1]:
%load_ext autoreload
%autoreload 2
import sys
#sys.path.append('/workspace/Embednet/')
#sys.path.append('/workspace/Graph_AE/')
import torch_geometric

In [1]:
%load_ext autoreload
%autoreload 2

#per la riproducibilità
%env CUBLAS_WORKSPACE_CONFIG=":4096:8"

import os
import itertools
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from IPython.display import Markdown as md
from multiprocessing import Pool
import time
from tqdm import tqdm
import networkx as nx
from networkx import stochastic_block_model

import torch_geometric
from graph_generation import GenerateGraph, perturb_nx_graph
from models import GCN, view_parameters, new_parameters, modify_parameters, Inits, new_parameters_linears
from train import Trainer, Dataset
from embedding import Embedding
from config_valid import Config, TrainingMode
import experiments
from experiments import Experiments, all_seeds
from plot_funcs import (plot_dim1, plot_dimN, plot_correlation_error, plot_metrics, plot_node_emb_1D, plot_node_emb_nD, scatter_node_emb, 
                        plot_graph_emb_1D, plot_graph_emb_nD, plot_data_degree_sequence, plot_corr_epoch, plot_ripetizioni_stesso_trial, 
                        plot_onlyloss_ripetizioni_stesso_trial,plot_onlyloss_ripetizioni_stesso_trial_superimposed, Data2Plot, plot_weights_multiple_hist)
from plot_model import plot_model
from utils import array_wo_outliers, plot_grafo, plot_grafo2
from Inspect import Inspect


import torch
from torch_geometric import nn
from torch_geometric.loader import DataLoader
device = torch.device('cuda')

from scipy import stats
from sklearn.metrics import log_loss

import pickle 
from plt_parameters import init_params, get_colors_to_cycle_rainbow8, get_colors_to_cycle_rainbowN
init_params()
rootsave = Path("output_plots/")

def make_video_parallel_static():
    experiments.graph_embedding_per_epoch = xp.trainer.graph_embedding_per_epoch
    experiments.node_embedding_per_epoch = xp.trainer.node_embedding_per_epoch
    experiments.dataset = xp.trainer.dataset
    experiments.loss_list = xp.trainer.test_loss_list
    experiments.exp_config = xp.trainer.config_class
    experiments.dataset_type = xp.trainer.gg.graphtype
    num_emb_neurons = xp.trainer.model.convs[-1].out_channels
    experiments.trainmode = xp.trainer.config_class.modo
    #experiments.num_classes = xp.trainer.config_class.num_classes
    experiments.embedding_dimension = num_emb_neurons
    #my_list = my_log_lista=list(range(20)) + list(range(20,100,4)) + list(range(100,500, 15)) + list(range(500, 5000, 40))
    nomefile = xp.make_video(skip=1, fromfiles=True, custom_list=True, seq_colors=True)
    return nomefile


def run_grid_w_gif(xp):
    nomifilesgif = []
    k = 0
    for c in xp.gc.configs:  
        print(f'Run {k + 1}\t\t exp name: {c.unique_train_name}')
        # all_seeds()
        xp.trainer.reinit_conf(c)
        xp.just_train()
        embedding_class = xp.embedding()
        num_emb_neurons = xp.trainer.model.convs[-1].out_channels
        trainmode = xp.trainer.config_class.modo
        embedding_class.get_metrics(num_emb_neurons, trainmode)    
        nomefile = make_video_parallel_static()
        nomifilesgif.append(nomefile)
        k += 1
    return nomifilesgif

env: CUBLAS_WORKSPACE_CONFIG=":4096:8"


2023-03-24 19:17:25.138111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-24 19:17:25.929904: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ImportError: cannot import name 'array_wo_outliers' from 'utils' (/workspace/Embednet/../Graph_AE/utils/__init__.py)

In [14]:
%cd Embednet

/workspace/Embednet


In [15]:
config_file = "configurations/Final1.yml"
num_nodi = 900
c = Config(config_file)
c.conf['graph_dataset']['Num_nodes'] = num_nodi
c.conf['graph_dataset']['list_exponents'] = [-2.5]
c.conf['model']['autoencoder'] = False
c.conf['model']['autoencoder_graph_ae'] = True
diz_trials = {'graph_dataset.ERmodel': [False], 'graph_dataset.confmodel': [False], 'graph_dataset.sbm': [False], 'graph_dataset.sbm': [True],
              'graph_dataset.Num_nodes': [[num_nodi], [num_nodi]*7, [num_nodi]*2],  # per lo SBM: num nodi * num comunità 
              'model.GCNneurons_per_layer': [#[1, 32, 16, len(c.conf['graph_dataset']['list_exponents'])],
                                            #[1, 32, 16, len(c.conf['graph_dataset']['list_p'])],
                                            #[1, 32, 16, len(c.conf['graph_dataset']['community_probs'])],
                                            [1, 128, 64]
                                           ],
             'model.init_weights': ['xavier_normal'],# 'eye'],
             'model.freezeGCNlayers': [False],
             'model.last_layer_dense': [False],
             } 

xp = Experiments(diz_trials=diz_trials, rootsave=rootsave, config_class=c)

2 configurazioni saltate su 3, farò i seguenti 1 training:
GraphType.SBM_Classi3_nodi900_grafiXtipo250_mode1_layers§1-128-64§_initw-xavier_normal_lr0.002_GCNfreezedFalse


In [20]:
xp.GS_simple_experiments()

Run 1 		 exp name: GraphType.SBM_Classi3_nodi900_grafiXtipo250_mode1_layers§1-128-64§_initw-xavier_normal_lr0.002_GCNfreezedFalse


IndexError: list index out of range

In [9]:
sys.path

['/workspace',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 'Embednet/',
 'Graph_AE/',
 '/workspace/Embednet/',
 '/workspace/Graph_AE/']

In [1]:
%cd ..

/workspace


In [2]:
from torch_geometric.data import DataLoader
from Graph_AE.utils.CustomDataSet import SelectGraph
from Graph_AE.utils.train_utils import train_cp
import torch
import argparse
import sys
#sys.path.append('/workspace/Embednet/')
sys.path.append('/workspace/Graph_AE/')

#%env 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128'
#%env CUBLAS_WORKSPACE_CONFIG=":4096:8"

In [3]:
parser = argparse.ArgumentParser(description='Global_Dict generator')
parser.add_argument('--d', type=str, default='FRANKENSTEIN', help="dataset name")
parser.add_argument('--m', type=str, default='MIAGAE', help="model name")
parser.add_argument('--device', type=str, default='cuda', help="cuda / cpu")
parser.add_argument('--batch', type=int, default=512, help="batch size")
parser.add_argument('--e', type=int, default=100, help="number of epochs")
parser.add_argument('--lr', type=float, default=1e-3, help="learning rate")
parser.add_argument('--model_dir', type=str, default="data/model/", help="path to save model")
parser.add_argument('--n_train', type=int, default=3000, help="number of samples for train set")
parser.add_argument('--n_test', type=int, default=1000, help="number of samples for test set")
parser.add_argument('--k', type=int, default=2, help="number of kernels")
parser.add_argument('--depth', type=int, default=3, help="depth of encoder and decoder")
parser.add_argument('--c_rate', type=float, default=0.8, help="compression ratio for each layer of encoder")
parser.add_argument('--shapes', type=str, default="64,64,64", help="shape of each layer in encoder")
#args = parser.parse_args()
sys.argv = ['--d','REDDIT-BINARY', '--n_train', '1500', '--e', '100']
args = parser.parse_args(sys.argv)
#args = parser.parse_args(['--d', 'COLORS-3'])
#args = parser.parse_args([])
args


Namespace(d='REDDIT-BINARY', m='MIAGAE', device='cuda', batch=512, e=100, lr=0.001, model_dir='data/model/', n_train=1500, n_test=1000, k=2, depth=3, c_rate=0.8, shapes='64,64,64')

In [1]:
args = {'d':'REDDIT-BINARY','m':'MIAGAE', 'device':'cuda', 'batch':512, 'e':100, 'lr':0.001, 'model_dir':'data/model/', 'n_train':1500, 'n_test':1000, 'k':2, 'depth':3, 'c_rate':0.8, 'shapes':'64,64,64'}

In [ ]:
device = torch.device(args.device)

num_epoch = args.e
batch_size = args.batch

SelectGraph.data_name = args.d
data_set = SelectGraph('data/' + SelectGraph.data_name)

#check sulle feature
dataset_mod = []
for d in data_set:
    if d.x is None:
        d.x = torch.ones([d.num_nodes], dtype=torch.float).unsqueeze(1) 
    dataset_mod.append(d)

    
train_set = DataLoader(dataset_mod[:args.n_train], batch_size=batch_size, shuffle=True)
test_set = DataLoader(dataset_mod[args.n_train:args.n_train + args.n_test], batch_size=batch_size, shuffle=False)

In [12]:
input_size = 1 # data_set.num_features
print(input_size)
shapes = list(map(int, args.shapes.split(",")))
if args.m == "MIAGAE":
    from classification.Graph_AE import Net
    model = Net(input_size, args.k, args.depth, [args.c_rate] * args.depth, shapes, device).to(device)
elif args.m == "UNet":
    from classification.UNet import Net
    model = Net(input_size, args.depth, args.c_rate, shapes, device).to(device)
elif args.m == "Gpool":
    from classification.Gpool_model import Net
    model = Net(input_size, args.depth, args.c_rate, shapes, device).to(device)
elif args.m == "SAGpool":
    from classification.SAG_model import Net
    model = Net(input_size, args.depth, [args.c_rate] * args.depth, shapes, device).to(device)
else:
    print("model not found")
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

1


In [13]:
train_cp(model, optimizer, device, train_set, test_set, num_epoch, args.model_dir, args.m)

OutOfMemoryError: CUDA out of memory. Tried to allocate 70.00 MiB (GPU 0; 23.65 GiB total capacity; 21.54 GiB already allocated; 63.69 MiB free; 22.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [2]:
l =  torch.nn.CrossEntropyLoss()

In [6]:
a = torch.randn(3, 5)
b = torch.randn(3, 5)
l(a,b)


tensor(-1.0684)

In [8]:
torch.nn.MSELoss()(a,b)

tensor(1.5887)

In [23]:
torch.ones([10]).unsqueeze(1).shape

torch.Size([10, 1])